In [1]:
#%pip install pyserial

In [2]:
import serial

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.signal
%matplotlib tk

In [36]:
fs = 20000

In [33]:
def tomaLecturas(puerto:str,cantidad:int,ancho:int=2,orden:str='little'):
    def procesaEscape(s:bytes):
        def proceso(s:bytes):
            i = 0
            while i < len(s):
                sk = s[i]
                match sk:
                    case b'\x0c':
                        pass
                    case b'\xdb':
                        if i+1 == len(s): break
                        i = i+1
                        match s[i]:
                            case b'\xdc':
                                yield b'\x0c'
                            case b'\xdd':
                                yield b'\xdb'
                            case _:
                                break
                    case _:
                        yield sk
                i += 1
        return bytes(proceso(s))
    def recibeBytes(puerto,cantidad):
        separador = b'\x0c'
        def generadorAuxiliar(puerto,cantidad):
            with serial.Serial(puerto) as s:
                cuenta = 0
                datos=s.read_until(separador)
                while cuenta < cantidad:
                    datos=s.read_until(separador)
                    if len(datos)<=1 : datos = s.read_until(separador)
                    datos = procesaEscape(datos)
                    yield from datos
                    cuenta += len(datos)
        return bytes(generadorAuxiliar(puerto,cantidad))
        
    datos = recibeBytes(puerto,cantidad*ancho)
    cantidad = len(datos)//ancho
    return np.array([ int.from_bytes(datos[ancho*i:ancho*(i+1)],orden,signed=False) for i in range(cantidad)])

In [41]:
lecturas = tomaLecturas("com24",fs)

In [ ]:
plt.plot(lecturas)


In [ ]:
plt.stem(*scipy.signal.periodogram(lecturas,fs,'hamming'))

In [16]:
with serial.Serial("com24") as s:
    d = s.read(1024)

In [ ]:
d.find(b'\x0c\x0c')

In [ ]:
d[127:].find(b'\x0c')

In [ ]:
d[126:]